In [5]:
#import libraries
!pip install kaggle
import kaggle
#download dataset using kaggle api --downloading anyone playstore dataset using old api
!kaggle datasets download arnikaer/googleplaystore -f googleplaystore.csv

Dataset URL: https://www.kaggle.com/datasets/arnikaer/googleplaystore
License(s): unknown
googleplaystore.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
#extract file from zip file
import zipfile
zip_ref=zipfile.ZipFile('googleplaystore.csv.zip')
zip_ref.extractall() #extract file to dir
zip_ref.close() #close file

In [3]:
# install and import pyspark 
#!pip install pyspark it is already installed

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import *


In [4]:
# Initialize SparkSession  generally in databricks it is not required  "\" we are using as line continuation
"""spark = SparkSession.builder.appName("GooglePlaystoreApp")\ # Assigns a name to your Spark job
    .master("local[*]") \             # Runs locally using all available CPU cores
    .config("spark.executor.memory", "2g") \  # Allocates 2 GB memory for each executor
    .getOrCreate()                    # Retrieves or creates a new Spark session
"""
spark = SparkSession.builder \
    .appName("GooglePlaystoreApp") \
    .getOrCreate()
#Create Dataframe
df=spark.read.load('googleplaystore.csv',format='csv',sep=',',header='true',escape='"',inferschema='true')

<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
C:\Users\khila\AppData\Local\Temp\ipykernel_18076\2106374959.py:2: SyntaxWarning: invalid escape sequence '\ '
  """spark = SparkSession.builder.appName("GooglePlaystoreApp")\ # Assigns a name to your Spark job


In [5]:
df.count()

10841

In [6]:
df.show(5)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|    7-Jan-18|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|   15-Jan-18|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    1-Aug-18|             1.2.4|4.0.3 and up|
|Sketch - Draw & P...|ART_AN

In [7]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [8]:
#Data cleaning steps
df=df.drop("size","Content Rating","Last Updated","Android Ver","Current Ver")

In [13]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [12]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [11]:
df=df.withColumn("Reviews",col("Reviews").cast(IntegerType()))\
.withColumn("Installs",regexp_replace(col("Installs"),"[^0-9]",""))\
.withColumn("Installs",col("Installs").cast(IntegerType()))\
.withColumn("Price",regexp_replace(col("Price"),"[$]",""))\
.withColumn("Price",col("Price").cast(IntegerType()))


In [15]:
df.show(5)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 5 rows



In [17]:
df.createOrReplaceTempView("apps")